In [236]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.26.0.3  317.43 KiB  16      100.0%            81b4a457-e16a-4a02-814d-f512b8a65a9d  rack1
UN  172.26.0.2  195.74 KiB  16      100.0%            3f9ad223-fdfc-4c9b-a8cd-0750da95d497  rack1
UN  172.26.0.4  197.6 KiB   16      100.0%            35f18065-50a7-4204-ae3c-847acea9bf88  rack1



In [237]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [238]:
cass.execute("DROP KEYSPACE IF EXISTS weather")

In [239]:
cass.execute("""
CREATE KEYSPACE IF NOT EXISTS weather
WITH REPLICATION = { 
   'class' : 'SimpleStrategy', 
   'replication_factor' : 3 
};
""")

In [240]:
cass.execute("USE weather")

In [241]:
cass.execute("CREATE TYPE station_record (tmin int, tmax int)")

In [242]:
cass.execute("""
create table stations(
    id TEXT,
    name TEXT STATIC,
    date DATE,
    record weather.station_record,
    PRIMARY KEY ((id), date) 
) WITH CLUSTERING ORDER BY (date ASC)
""")

In [28]:
#q1
#What is the Schema of stations?
cass.execute("describe table weather.stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [29]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ed41c74d-08a0-49dd-82c1-25f34b98018a;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [68]:
df = spark.read.text("ghcnd-stations.txt")

In [69]:
df.take(10)

[Row(value='ACW00011604  17.1167  -61.7833   10.1    ST JOHNS COOLIDGE FLD                       '),
 Row(value='ACW00011647  17.1333  -61.7833   19.2    ST JOHNS                                    '),
 Row(value='AE000041196  25.3330   55.5170   34.0    SHARJAH INTER. AIRP            GSN     41196'),
 Row(value='AEM00041194  25.2550   55.3640   10.4    DUBAI INTL                             41194'),
 Row(value='AEM00041217  24.4330   54.6510   26.8    ABU DHABI INTL                         41217'),
 Row(value='AEM00041218  24.2620   55.6090  264.9    AL AIN INTL                            41218'),
 Row(value='AF000040930  35.3170   69.0170 3366.0    NORTH-SALANG                   GSN     40930'),
 Row(value='AFM00040938  34.2100   62.2280  977.2    HERAT                                  40938'),
 Row(value='AFM00040948  34.5660   69.2120 1791.3    KABUL INTL                             40948'),
 Row(value='AFM00040990  31.5000   65.8500 1010.0    KANDAHAR AIRPORT                      

In [70]:
from pyspark.sql.functions import col, expr, rtrim

In [71]:
df2 = df.withColumn("ID", expr("substring(value, 1, 11)")).withColumn("STATE", expr("substring(value, 39, 2)")).withColumn("NAME", rtrim(expr("substring(value, 42, 30)"))).drop("value")

In [72]:
df2.take(10)

[Row(ID='ACW00011604', STATE='  ', NAME='ST JOHNS COOLIDGE FLD'),
 Row(ID='ACW00011647', STATE='  ', NAME='ST JOHNS'),
 Row(ID='AE000041196', STATE='  ', NAME='SHARJAH INTER. AIRP'),
 Row(ID='AEM00041194', STATE='  ', NAME='DUBAI INTL'),
 Row(ID='AEM00041217', STATE='  ', NAME='ABU DHABI INTL'),
 Row(ID='AEM00041218', STATE='  ', NAME='AL AIN INTL'),
 Row(ID='AF000040930', STATE='  ', NAME='NORTH-SALANG'),
 Row(ID='AFM00040938', STATE='  ', NAME='HERAT'),
 Row(ID='AFM00040948', STATE='  ', NAME='KABUL INTL'),
 Row(ID='AFM00040990', STATE='  ', NAME='KANDAHAR AIRPORT')]

In [73]:
df2.write.saveAsTable("ID_STATE_NAME_table", mode="overwrite")
df = spark.sql("""
SELECT *
FROM ID_STATE_NAME_table
WHERE STATE = "WI"
""")

In [74]:
# count = df.collect()[0]['count']
# count

In [75]:
weather_list = df.collect()

In [142]:
for row in weather_list:
    cass.execute("""
        INSERT INTO stations (ID, NAME)
        VALUES (%s, %s)
        """,(row.ID, row.NAME))

In [143]:
cass.execute("SELECT COUNT(*) FROM weather.stations").one()[0]

1313

In [78]:
#q2
#What is the name corresponding to station ID USW00014837?
cass.execute("""
    SELECT NAME 
    FROM weather.stations 
    WHERE ID = 'USW00014837'
""").one()[0]

'MADISON DANE CO RGNL AP'

In [87]:
#q3
#what is the token for the USC00470273 station?
token_0273 = cass.execute("""
    SELECT TOKEN(ID)
    FROM weather.stations 
    WHERE ID = 'USW00014837'
""").one()[0]

In [83]:
import subprocess
output = subprocess.check_output(['nodetool', 'ring'])
output

b'\nDatacenter: datacenter1\n==========\nAddress          Rack        Status State   Load            Owns                Token                                       \n                                                                                9159087601823522902                         \n172.26.0.4       rack1       Up     Normal  192.43 KiB      100.00%             -8836231418056521039                        \n172.26.0.3       rack1       Up     Normal  194.08 KiB      100.00%             -8477132856236156023                        \n172.26.0.2       rack1       Up     Normal  194.06 KiB      100.00%             -8196359361193816454                        \n172.26.0.3       rack1       Up     Normal  194.08 KiB      100.00%             -7659460923623805228                        \n172.26.0.2       rack1       Up     Normal  194.06 KiB      100.00%             -7351982713702654696                        \n172.26.0.4       rack1       Up     Normal  192.43 KiB      100.00%          

In [86]:
output_str = output.decode('utf-8')
lines = output_str.split('\n')
tokens = []
for line in lines:
    parts = line.split()
    if len(parts) > 1 and parts[-1].lstrip('-').isdigit(): # without lstrip, token with leading "-" will be removed
        tokens.append(int(parts[-1]))
tokens

[-8836231418056521039,
 -8477132856236156023,
 -8196359361193816454,
 -7659460923623805228,
 -7351982713702654696,
 -6920100163552343935,
 -6583705042398225688,
 -6174032324647428096,
 -5911714483800071456,
 -5847789349687332047,
 -5223791281012818115,
 -4840213867460038644,
 -4768416542239730390,
 -4204056237459408519,
 -3822362462717611628,
 -3524896536559138416,
 -3004690530105260834,
 -2557532293480602918,
 -1928934648879681294,
 -1495245955457340713,
 -1193018956168787654,
 -967527740966704280,
 -438024586690667210,
 -113646148721185997,
 358962329284396099,
 651495554730339016,
 1129873856959405975,
 1629641007246453092,
 2097238100037941474,
 2545076732869639646,
 2887443734399612232,
 3408275308070040220,
 3687242652551840112,
 4172345351568715193,
 4504317208304978509,
 4996215887986982353,
 5306265948248883409,
 5595614318997190768,
 6015737482411237822,
 6284411400764997485,
 6439990966488352526,
 6980391776839723544,
 7314801238241361129,
 7880259196390935764,
 806304570158

In [99]:
#q4
#what is the first vnode token in the ring following the token for USC00470273?
for i, node in enumerate(tokens):
    if token_0273 > tokens[i] and i + 1 == len(tokens): # wrapping around
        ans = tokens[0]
        break
    elif token_0273 < tokens[i+1]:
        ans = tokens[i+1]
        break
ans

8063045701588584613

In [100]:
!unzip records.zip

Archive:  records.zip
   creating: records.parquet/
  inflating: records.parquet/part-00000-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
 extracting: records.parquet/._SUCCESS.crc  
  inflating: records.parquet/part-00002-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
  inflating: records.parquet/part-00001-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
  inflating: records.parquet/part-00003-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
 extracting: records.parquet/.part-00003-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/_SUCCESS  
 extracting: records.parquet/.part-00000-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/.part-00001-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/.part-00002-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  


In [110]:
df_record = spark.read.parquet("records.parquet")

In [195]:
from pyspark.sql import functions as F
df2_record = (df_record.groupBy("station", "date")
                      .pivot("element", ["TMAX","TMIN"])
                      .agg(F.first("value"))
                      .orderBy("station"))

In [196]:
df2_record.show(10)

+-----------+--------+------+------+
|    station|    date|  TMAX|  TMIN|
+-----------+--------+------+------+
|USR0000WDDG|20220806| 317.0| 217.0|
|USR0000WDDG|20220924| 161.0|  94.0|
|USR0000WDDG|20220628| 283.0| 161.0|
|USR0000WDDG|20220130| -33.0|-117.0|
|USR0000WDDG|20220919| 278.0| 139.0|
|USR0000WDDG|20220414|  50.0| -17.0|
|USR0000WDDG|20220629| 306.0| 150.0|
|USR0000WDDG|20221114|  17.0| -61.0|
|USR0000WDDG|20220712| 289.0| 156.0|
|USR0000WDDG|20220202|-106.0|-150.0|
+-----------+--------+------+------+
only showing top 10 rows



In [ ]:
# cass.execute("""
# create table stations(
#     id TEXT,
#     name TEXT STATIC,
#     date DATE,
#     record weather.station_record,
#     PRIMARY KEY ((id), date) 
# ) WITH CLUSTERING ORDER BY (date ASC)
# """)

In [192]:
import grpc
import station_pb2 
import station_pb2_grpc 


rows = df2_record.collect()
channel = grpc.insecure_channel(f"localhost:5440") 
stub = station_pb2_grpc.StationStub(channel)

for row in rows:
    year = row.date[:4]
    month = row.date[4:6]
    day = row.date[6:]    
    request = station_pb2.RecordTempsRequest(
        station=row.station,
        date=str(year + "-" + month + "-" + day),
        tmin=int(row.TMIN),
        tmax=int(row.TMAX)
    )
    try:
        response = stub.RecordTemps(request)
        # print(response)
    except grpc.RpcError as e:
        print("gRPC call failed: {}".format(e))

In [193]:
result = cass.execute("""
SELECT *
FROM stations
WHERE id = 'USW00014837'
LIMIT 5
""")
for row in result:
    print(row)

Row(id='USW00014837', date=Date(18993), name='MADISON DANE CO RGNL AP', record=station_record(tmin=-99, tmax=-32))
Row(id='USW00014837', date=Date(18994), name='MADISON DANE CO RGNL AP', record=station_record(tmin=-166, tmax=-82))
Row(id='USW00014837', date=Date(18995), name='MADISON DANE CO RGNL AP', record=station_record(tmin=-177, tmax=-66))
Row(id='USW00014837', date=Date(18996), name='MADISON DANE CO RGNL AP', record=station_record(tmin=-88, tmax=-5))
Row(id='USW00014837', date=Date(18997), name='MADISON DANE CO RGNL AP', record=station_record(tmin=-116, tmax=-5))


In [248]:
#q5
#what is the max temperature ever seen for station USW00014837?
request = station_pb2.StationMaxRequest(station='USW00014837')
response = stub.StationMax(request)
response

error: "Error from server: code=1000 [Unavailable exception] message=\"Cannot achieve consistency level THREE\" info={\'consistency\': \'THREE\', \'required_replicas\': 3, \'alive_replicas\': 2}"

In [206]:
df = (spark.read.format("org.apache.spark.sql.cassandra")
      .option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3")
      .option("keyspace", "weather")
      .option("table", "stations")
      .load())

In [210]:
df.createOrReplaceTempView("stations")

In [212]:
#q6
#what tables/views are available in the Spark catalog?
spark.sql("DROP TABLE IF EXISTS id_state_name_table")
spark.catalog.listTables()

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [231]:
#q7
#what is the average difference between tmax and tmin, for each of the four stations that have temperature records?
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

def get_tmax(record):
    return record.tmax if record is not None else None

def get_tmin(record):
    return record.tmin if record is not None else None

get_tmax_udf = udf(get_tmax, IntegerType())
get_tmin_udf = udf(get_tmin, IntegerType())

spark.udf.register("get_tmax", get_tmax_udf)
spark.udf.register("get_tmin", get_tmin_udf)

diff_df = spark.sql("""
SELECT id, AVG(get_tmax(record) - get_tmin(record)) as diff
FROM stations
WHERE id = "USW00014839"
OR id = "USR0000WDDG"
OR id = "USW00014837"
OR id = "USW00014898"
GROUP BY id
""")
rows = diff_df.collect()
result_dict = {row['id']: row['diff'] for row in rows}
result_dict

23/11/17 10:35:29 WARN SimpleFunctionRegistry: The function get_tmax replaced a previously registered function.
23/11/17 10:35:29 WARN SimpleFunctionRegistry: The function get_tmin replaced a previously registered function.


{'USW00014839': 89.6986301369863,
 'USW00014837': 105.62739726027397,
 'USR0000WDDG': 102.06849315068493,
 'USW00014898': 102.93698630136986}

23/11/17 10:37:11 WARN ChannelPool: [s0|p6-db-2/172.26.0.3:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=bc3f6eab-0e53-4213-9f2a-176ee0fb5e72, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700164237797}): failed to send request (java.nio.channels.NotYetConnectedException))
23/11/17 10:37:19 WARN ChannelPool: [s0|p6-db-2/172.26.0.3:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=bc3f6eab-0e53-4213-9f2a-176ee0fb5e72, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700164237797}): failed to send request (java.nio.channels.NotYetConnectedException))
23/11/17 10:37:28 WARN ChannelPool: [s0|p6-db-2/

In [244]:
#q8
#what does nodetool status output?
! nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.26.0.3  317.43 KiB  16      100.0%            81b4a457-e16a-4a02-814d-f512b8a65a9d  rack1
UN  172.26.0.2  195.74 KiB  16      100.0%            3f9ad223-fdfc-4c9b-a8cd-0750da95d497  rack1
UN  172.26.0.4  182.07 KiB  16      100.0%            35f18065-50a7-4204-ae3c-847acea9bf88  rack1



In [249]:
#q9
#if you make a StationMax RPC call, what does the error field contain in StationMaxReply reply?
request = station_pb2.StationMaxRequest(station='USW00014837')
response = stub.StationMax(request)
response

23/11/17 10:47:09 WARN ChannelPool: [s0|p6-db-2/172.26.0.3:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=bc3f6eab-0e53-4213-9f2a-176ee0fb5e72, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700164237797}): failed to send request (java.nio.channels.NotYetConnectedException))


error: "Error from server: code=1200 [Coordinator node timed out waiting for replica nodes\' responses] message=\"Operation timed out - received only 2 responses.\" info={\'consistency\': \'THREE\', \'required_responses\': 3, \'received_responses\': 2}"

23/11/17 10:47:16 WARN ChannelPool: [s0|p6-db-2/172.26.0.3:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=bc3f6eab-0e53-4213-9f2a-176ee0fb5e72, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700164237797}): failed to send request (java.nio.channels.NotYetConnectedException))
23/11/17 10:47:24 WARN ChannelPool: [s0|p6-db-2/172.26.0.3:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=bc3f6eab-0e53-4213-9f2a-176ee0fb5e72, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700164237797}): failed to send request (java.nio.channels.NotYetConnectedException))
23/11/17 10:47:32 WARN ChannelPool: [s0|p6-db-2/

In [250]:
#q10
#if you make a RecordTempsRequest RPC call, what does error contain in the RecordTempsReply reply?
request = station_pb2.RecordTempsRequest(
    station="UWMADISON",
    date=str("2023-4-20"),
    tmin=int(0),
    tmax=int(100)
)

response = stub.RecordTemps(request)
response

23/11/17 10:50:28 WARN ChannelPool: [s0|p6-db-2/172.26.0.3:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=bc3f6eab-0e53-4213-9f2a-176ee0fb5e72, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700164237797}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
